In [3]:
import ee
import geemap
import os

In [4]:
Map = geemap.Map(center=(-0.838534, 35.223872), zoom=13)
Map

Map(center=[-0.838534, 35.223872], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
parcels = ee.FeatureCollection("projects/ee-koros-projects/assets/Koimeret_Registration_section")

In [6]:
image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(parcels)
    .filterDate('2016-01-01', '2016-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

Map.centerObject(parcels, 8)
Map.addLayer(image.clip(parcels), vis_params, "Landsat-8")
Map

Map(bottom=1053761.0, center=[-0.8356866055744563, 35.22486634901747], controls=(WidgetControl(options=['posit…

In [7]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2016-02-16'

In [8]:
image.get('CLOUD_COVER').getInfo()

0.01

In [16]:
# create function to crop with table boundaries
def table_bounds(image):
  # Crop by table extension
  return image.clip(parcels)


In [ ]:
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m").map(table_bounds)
# Define a dictionary which will be used to make legend and visualize image on map
dict = {
  "names": [
    "Water",
    "Trees",
    "Grass",
    "Flooded Vegetation",
    "Crops",
    "Scrub/Shrub",
    "Built Area",
    "Bare Ground",
    "Snow/Ice",
    "Clouds"
  ],
  "colors": [
    "#1A5BAB",
    "#358221",
    "#A7D282",
    "#87D19E",
    "#FFDB5C",
    "#EECFA8",
    "#ED022A",
    "#EDE9E4",
    "#F2FAFF",
    "#C8C8C8"
  ]}


  # Get the list of palette colors and class names from the image.
palette = dict['colors']
names = dict['names']


# Add image to the map
Map.addLayer(esri_lulc10.first(), {'min':1, 'max':10, 'palette':dict['colors']}, 'ESRI LULC 10m')
Map


Map(bottom=17146.0, center=[-5.069057826784033, 32.77221679687501], controls=(WidgetControl(options=['position…

In [9]:
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m");

In [11]:
# Define a dictionary which will be used to make legend and visualize image on map
dict = {
  "names": [
    "Water",
    "Trees",
    "Grass",
    "Flooded Vegetation",
    "Crops",
    "Scrub/Shrub",
    "Built Area",
    "Bare Ground",
    "Snow/Ice",
    "Clouds"
  ],
  "colors": [
    "#1A5BAB",
    "#358221",
    "#A7D282",
    "#87D19E",
    "#FFDB5C",
    "#EECFA8",
    "#ED022A",
    "#EDE9E4",
    "#F2FAFF",
    "#C8C8C8"
  ]}

# Get the list of palette colors and class names from the image.
palette = dict['colors']
names = dict['names']
# Add image to the map
Map.addLayer(esri_lulc10.mosaic().clip(image.geometry()), {min:1, max:10, palette:dict['colors']}, 'ESRI LULC 10m')
Map

TypeError: unhashable type: 'list'

In [ ]:
nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())
Map.addLayer(nlcd, {}, 'NLCD')
Map